In [1]:
import requests
import json
import warnings
import pandas as pd
from datetime import datetime, timedelta, date
from pprint import pprint

# OBTENCIÓN DE DATOS DE INTERCAMBIO 

In [2]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [7]:
def intercambio_datos(lang, restaDia, input_año):

    now = datetime.now()
    ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')
    hoy = now.replace(year = input_año).strftime('%Y-%m-%d')

    query = f"start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day"


    endpoint = f"https://apidatos.ree.es/{lang}/datos/intercambios/todas-fronteras-programados?{query}"
    response = requests.get(url = endpoint, headers = headers)
    data = response.json()


    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()
    lista_dias = list()
    lista_meses = list()
    lista_años = list()


    for dato in data['included']: 
        for info in dato['attributes']['content']:
            tipo = info['type']

            nombre = info['groupId']


            for i in info['attributes']['values']:
                valor = i['value']

                porcentaje = i['percentage']

                fecha = i['datetime']
                fecha = pd.to_datetime(fecha)
                dia = fecha.strftime('%d')
                mes = fecha.strftime('%m')
                año = fecha.strftime('%Y')

                
                lista_nombres.append(nombre)
                lista_tipos.append(tipo)
                lista_valores.append(valor)
                lista_porcentajes.append(porcentaje)
                lista_fechas.append(fecha)
                lista_dias.append(dia)
                lista_meses.append(mes)
                lista_años.append(año)


    df_intercambio = pd.DataFrame()
    df_intercambio['nombre']               = lista_nombres
    df_intercambio['tipo de intercambio']      = lista_tipos
    df_intercambio['Valores']              = lista_valores
    df_intercambio["Porcentaje"]           = lista_porcentajes
    df_intercambio["Fecha actualización"]  = lista_fechas
    df_intercambio["dia"]                  = lista_dias               
    df_intercambio["mes"]                  = lista_meses               
    df_intercambio["año"]                  = lista_años               


    return df_intercambio  

In [5]:
lang    =  input('¿En español (es) o en inglés (en)?') 
#category = input('¿Balance, demanda, generación o intercambio?') 
#widget   = input('¿Qué widget quieres?')


restaDia = float(input("¿Cuantos días atrás?: ______________"))
input_año = int(input("¿Qué año?: ______________"))

#balance_datos(lang, category, widget, restaDia, input_año)

In [ ]:
#intercambio_datos(lang, restaDia, input_año).to_csv('intercambio_electrico.csv')

In [8]:
df_intercambio = intercambio_datos(lang, restaDia, input_año)
df_intercambio

,nombre,tipo de intercambio,Valores,Porcentaje,Fecha actualización,dia,mes,año
0,Francia,Exportación,-2795.300,0.045989,2024-09-24 00:00:00+02:00,24,09,2024
1,Francia,Exportación,-11818.600,0.196296,2024-09-25 00:00:00+02:00,25,09,2024
2,Francia,Exportación,-26348.100,0.488307,2024-09-26 00:00:00+02:00,26,09,2024
3,Francia,Exportación,-17153.200,0.271578,2024-09-27 00:00:00+02:00,27,09,2024
4,Francia,Exportación,-24870.267,0.374056,2024-09-28 00:00:00+02:00,28,09,2024
...,...,...,...,...,...,...,...,...
78,Andorra,saldo,-941.700,1.000000,2024-09-27 00:00:00+02:00,27,09,2024
79,Andorra,saldo,-503.000,1.000000,2024-09-28 00:00:00+02:00,28,09,2024
80,Andorra,saldo,-376.000,1.000000,2024-09-29 00:00:00+02:00,29,09,2024
81,Andorra,saldo,-237.000,1.000000,2024-09-30 00:00:00+02:00,30,09,2024


In [9]:
df_intercambio.to_csv('intercambio_electrico.csv')